In [1]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import gc

from tqdm import tqdm
import itertools

from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
pd.options.mode.copy_on_write = True

In [5]:
dir_data = '../data/netflow/parquet/original/'
dir_model = 'model/'

In [6]:
emb_prefix = "emb_wo_port_"

In [7]:
netflows = ['NF-BoT-IoT_chunks',
           'NF-ToN-IoT_chunks',
           'NF-UNSW-NB15_chunks',
           'NF-UQ-NIDS_chunks',
           'NF-CSE-CIC-IDS2018_chunks',
           'NF-BoT-IoT-v2_chunks',
           'NF-ToN-IoT-v2_chunks',
           'NF-UNSW-NB15-v2_chunks',
           'NF-UQ-NIDS-v2_chunks',
           'NF-CSE-CIC-IDS2018-v2_chunks']

In [6]:
df_test = pd.read_parquet(dir_data + emb_prefix + 'Attack-2_chunks')

for i, nf in enumerate(netflows):
    print ("\n==== '" + emb_prefix+nf + "' ====")
    
    n_est = [20, 50, 100, 150]
    cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
    params = list(itertools.product(n_est, cont))
    score = -1
    bs = None
    PATH = dir_data + emb_prefix + nf
    df_train = pd.read_parquet(PATH)
    
    benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label"])
    normal_train_samples = df_train.drop(columns=["Label"])

    train_labels = df_train["Label"]
    test_labels = df_test["Label"]

    test_samples = df_test.drop(columns=["Label"])
    
    
    for n_est, con in tqdm(params):
        clf_if = IsolationForest(n_estimators=n_est, contamination=con)
        clf_if.fit(benign_train_samples)
        y_pred = clf_if.predict(test_samples)
        test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

        f1 = f1_score(test_labels, test_pred, average='macro')

        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est,
                           "con": con
                    }
            bs = test_pred
        del clf_if
        gc.collect()


    print(best_params)
    print(score)
    print(classification_report(test_labels, bs, digits=4))


==== 'emb_wo_port_NF-BoT-IoT_chunks' ====


100%|███████████████████████████████████████████| 24/24 [00:04<00:00,  5.60it/s]


{'n_estimators': 50, 'con': 0.01}
0.6405259965887186
              precision    recall  f1-score   support

           0     0.4582    0.4705    0.4643      9056
           1     0.8205    0.8131    0.8168     26960

    accuracy                         0.7270     36016
   macro avg     0.6393    0.6418    0.6405     36016
weighted avg     0.7294    0.7270    0.7281     36016


==== 'emb_wo_port_NF-ToN-IoT_chunks' ====


100%|███████████████████████████████████████████| 24/24 [00:08<00:00,  2.73it/s]


{'n_estimators': 20, 'con': 0.04}
0.7836857055152637
              precision    recall  f1-score   support

           0     0.5626    0.9901    0.7175      9056
           1     0.9955    0.7414    0.8499     26960

    accuracy                         0.8039     36016
   macro avg     0.7791    0.8657    0.7837     36016
weighted avg     0.8867    0.8039    0.8166     36016


==== 'emb_wo_port_NF-UNSW-NB15_chunks' ====


...

{'n_estimators': 20, 'con': 0.05}
0.42809959349593496
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      9056
           1     0.7486    1.0000    0.8562     26960

    accuracy                         0.7486     36016
   macro avg     0.3743    0.5000    0.4281     36016
weighted avg     0.5603    0.7486    0.6409     36016


==== 'emb_wo_port_NF-UQ-NIDS-v2_chunks' ====


100%|███████████████████████████████████████████| 24/24 [03:06<00:00,  7.78s/it]


{'n_estimators': 150, 'con': 0.001}
0.7854097626249152
              precision    recall  f1-score   support

           0     0.5744    0.9402    0.7131      9056
           1     0.9744    0.7660    0.8577     26960

    accuracy                         0.8098     36016
   macro avg     0.7744    0.8531    0.7854     36016
weighted avg     0.8738    0.8098    0.8214     36016


==== 'emb_wo_port_NF-CSE-CIC-IDS2018-v2_chunks' ====


100%|███████████████████████████████████████████| 24/24 [03:33<00:00,  8.91s/it]

{'n_estimators': 150, 'con': 0.01}
0.6051448926477966
              precision    recall  f1-score   support

           0     0.3916    0.4798    0.4312      9056
           1     0.8110    0.7496    0.7791     26960

    accuracy                         0.6818     36016
   macro avg     0.6013    0.6147    0.6051     36016
weighted avg     0.7055    0.6818    0.6916     36016



In [8]:
df_train = pd.DataFrame()
for i, nf in enumerate(netflows):
    #print ("\n==== '" + emb_prefix+nf + "' ====")
    
    PATH = dir_data + emb_prefix + nf
    df_train = pd.concat([df_train, pd.read_parquet(PATH)])

df_train = df_train.drop_duplicates()
filtered_df = df_train.groupby(list(df_train.columns)[:-1])[list(df_train.columns)[-1]].transform('nunique') != 1
df_train = df_train[~filtered_df]
df_train = df_train.sample(frac=1).reset_index(drop=True)

In [41]:
df_test = pd.read_parquet(dir_data + emb_prefix + 'Attack-2_chunks')

In [ ]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label"])
normal_train_samples = df_train.drop(columns=["Label"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label"])


for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

In [7]:
emb_prefix = "emb_port_"

In [8]:
for i, nf in enumerate(netflows):
    print ("\n==== '" + emb_prefix+nf + "' ====")
    
    n_est = [20, 50, 100, 150]
    cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
    params = list(itertools.product(n_est, cont))
    score = -1
    bs = None
    PATH = dir_data + emb_prefix + nf
    df_train = pd.read_parquet(PATH)
    
    benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label"])
    normal_train_samples = df_train.drop(columns=["Label"])

    train_labels = df_train["Label"]
    test_labels = df_test["Label"]

    test_samples = df_test.drop(columns=["Label"])
    
    
    for n_est, con in tqdm(params):
        clf_if = IsolationForest(n_estimators=n_est, contamination=con)
        clf_if.fit(benign_train_samples)
        y_pred = clf_if.predict(test_samples)
        test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

        f1 = f1_score(test_labels, test_pred, average='macro')

        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est,
                           "con": con
                    }
            bs = test_pred
        del clf_if
        gc.collect()


    print(best_params)
    print(score)
    print(classification_report(test_labels, bs, digits=4))


==== 'emb_port_NF-BoT-IoT_chunks' ====


100%|███████████████████████████████████████████| 24/24 [00:22<00:00,  1.08it/s]


{'n_estimators': 20, 'con': 0.04}
0.7554763535935455
              precision    recall  f1-score   support

           0     0.3625    0.9488    0.5246      9104
           1     0.9992    0.9739    0.9864    581118

    accuracy                         0.9735    590222
   macro avg     0.6808    0.9613    0.7555    590222
weighted avg     0.9894    0.9735    0.9792    590222


==== 'emb_port_NF-ToN-IoT_chunks' ====


100%|███████████████████████████████████████████| 24/24 [00:27<00:00,  1.13s/it]


{'n_estimators': 20, 'con': 0.01}
0.4961138525107996


...

{'n_estimators': 20, 'con': 0.04}
0.4961138525107996


/Users/mammadli/miniconda3/envs/py_jop/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mammadli/miniconda3/envs/py_jop/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mammadli/miniconda3/envs/py_jop/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      9104
           1     0.9846    1.0000    0.9922    581118

    accuracy                         0.9846    590222
   macro avg     0.4923    0.5000    0.4961    590222
weighted avg     0.9694    0.9846    0.9769    590222


==== 'emb_port_NF-CSE-CIC-IDS2018-v2_chunks' ====


100%|███████████████████████████████████████████| 24/24 [02:18<00:00,  5.76s/it]


{'n_estimators': 20, 'con': 0.05}
0.4974992878695194
              precision    recall  f1-score   support

           0     0.0114    0.0031    0.0048      9104
           1     0.9846    0.9958    0.9902    581118

    accuracy                         0.9805    590222
   macro avg     0.4980    0.4994    0.4975    590222
weighted avg     0.9695    0.9805    0.9750    590222



In [42]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label"])
normal_train_samples = df_train.drop(columns=["Label"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label"])


for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|███████████████████████████████████████████| 24/24 [03:55<00:00,  9.80s/it]

{'n_estimators': 50, 'con': 0.05}
0.6602252521665287
              precision    recall  f1-score   support

           0     0.4319    0.9402    0.5919      9056
           1     0.9668    0.5846    0.7286     26960

    accuracy                         0.6740     36016
   macro avg     0.6993    0.7624    0.6602     36016
weighted avg     0.8323    0.6740    0.6942     36016



In [43]:
df_train = df_train.sample(frac=1).reset_index(drop=True)

In [46]:
df_train = df_train_

In [37]:
sample_size = 100000
df_train = df_train.sample(n=sample_size, random_state=1)

In [44]:
l = int(0.9*df_train.shape[0])
X = df_train.drop(columns=["Label"])
y = df_train[["Label"]]

X_train = X[:l]
y_train = y[:l]

X_val = X[l:]
y_val = y[l:]

In [46]:
rf_classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0, verbose=1)
rf_classifier.fit(X_train, y_train)

/Users/mammadli/miniconda3/envs/py_jop/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0,
                       verbose=1)

In [47]:
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)

/Users/mammadli/miniconda3/envs/py_jop/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [51]:
y_val_pred = rf_classifier.predict(X_val)
cm = confusion_matrix(y_val, y_val_pred)
acc = accuracy_score(y_val, y_val_pred)
print(cm, acc)

[[412107   1403]
 [  2319    246]] 0.9910544973862885


In [49]:
y_val_pred = knn_classifier.predict(X_val.values)
cm = confusion_matrix(y_val, y_val_pred)
acc = accuracy_score(y_val, y_val_pred)
print(cm, acc)

/Users/mammadli/miniconda3/envs/py_jop/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


[[412474   1036]
 [  2228    337]] 0.9921552604698672


In [50]:
X_test = df_test.drop(columns=["Label"])
y_test = df_test[["Label"]]

In [52]:
y_pred = rf_classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(cm, acc)

[[ 9056     0]
 [26958     2]] 0.251499333629498


In [53]:
y_pred = knn_classifier.predict(X_test.values)
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(cm, acc)

/Users/mammadli/miniconda3/envs/py_jop/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


[[ 4536  4520]
 [15785 11175]] 0.43622278987116836
